In [ ]:
*clojure-version*

In [ ]:
(require '(incanter core stats charts io))
(require '[clojure.repl :refer :all])

In [ ]:
;(-> (incanter.stats/sample-normal 10000)
;    incanter.charts/histogram
;    (.createBufferedImage 600 400))

In [ ]:
(require '[metaprob.state :as state])
(require '[metaprob.trace :as trace])
(require '[metaprob.sequence :as sequence])
(require '[metaprob.builtin-impl :as impl])

(require '[metaprob.syntax :refer :all])
(require '[metaprob.builtin :refer :all])
(require '[metaprob.prelude :refer :all])
(require '[metaprob.distributions :refer :all])
(require '[metaprob.interpreters :refer :all])
(require '[metaprob.inference :refer :all])

(require '[metaprob.infer :as infer])
(require '[metaprob.compositional :as comp])
(require '[metaprob.examples.gaussian :refer :all])

In [ ]:
(define example-2D
  (gen []
    (define x (gaussian 0 1))
    (define y (gaussian x 1))
    (+ x y)))

In [ ]:
(define get-trace
    (gen [proc inputs]
         (define out (empty-trace))
         (infer :procedure proc
                :inputs inputs
                :output-trace out)
         out))

(define get-example-2D-traces
    (gen [n]
         (take n (repeatedly (gen [] (get-trace example-2D nil))))))

(define get-xs
    (gen [traces]
         (map (gen [rho] (trace-get rho (addr 0 "x" "gaussian"))) traces)))

(define get-ys
    (gen [traces]
         (map (gen [rho] (trace-get rho (addr 1 "y" "gaussian"))) traces)))

In [ ]:
(define make-example-2D-trace 
    (gen [x y]
        (trace 0 (** (trace "x" (** (trace "gaussian" x))))
               1 (** (trace "y" (** (trace "gaussian" y))))
               )))

(define score-example-2D
    (gen [x y]
         (get (infer :procedure example-2D
                     :inputs nil
                     :target-trace (make-example-2D-trace x y))
              2)))

In [ ]:
(.createBufferedImage
    (incanter.charts/heat-map score-example-2D
                              -3 3 -3 3
                              :title "Bivariate Gaussian --- exact density")
    300 300)

In [ ]:
(define example-2D-traces (get-example-2D-traces 1000))
(.createBufferedImage
    (incanter.charts/scatter-plot
        (get-xs example-2D-traces)
        (get-ys example-2D-traces)
        :title "Bivariate Gaussian generative procedure --- sampled traces")
    500 500)

In [ ]:
(define make-example-2D-density-y-given-x
    (gen [xval]
         (gen [y]
             (get (infer :procedure example-2D
                         :inputs nil
                         :target-trace (make-example-2D-trace xval y))
                  2))
         ))

(define make-example-2D-sampler-y-given-x
    (gen [xval K]
         (gen []
             (importance-resampling example-2D nil
                                    (trace 0 (** (trace "x" (** (trace "gaussian" xval)))))
                                    K))
             ))

In [ ]:
(.createBufferedImage (incanter.charts/function-plot (make-example-2D-density-y-given-x 1.0)
                                                     -5 5)
                      300 300)

In [ ]:
(define example2D-traces-given-x
    (take 1000 (repeatedly (make-example-2D-sampler-y-given-x 1.0 10))))

(.createBufferedImage
    (incanter.charts/histogram
        (get-ys example2D-traces-given-x)
        :title "Bivariate Gaussian generative procedure --- sampled traces given x = 1.0"
        )
    300 300)

In [ ]:
(.createBufferedImage
    (incanter.charts/function-plot
        (make-example-2D-density-y-given-x 1.0)
        -2 4
        :title "Bivariate Gaussian generative procedure --- density given x = 1.0"
        )
    300 300)

In [ ]:
(define example2D-traces-given-x-set2
    (take 1000 (repeatedly (make-example-2D-sampler-y-given-x 0.0 10))))

(.createBufferedImage
    (incanter.charts/histogram
        (get-ys example2D-traces-given-x-set2)
        :title "Bivariate Gaussian generative procedure --- sampled traces given x = 0.0"
        )
    300 300)

In [ ]:
(.createBufferedImage
    (incanter.charts/function-plot
        (make-example-2D-density-y-given-x 0.0)
        -3 3
        :title "Bivariate Gaussian generative procedure --- density given x = 0.0"
        )
    300 300)